In [0]:
from pyspark.sql.functions import *

## Dynamic Parameters
**file_path** - subfolder of source location that the file is located <br>
**file_name** - name of the file <br>
**file_format** - format of the file from source location

In [0]:
dbutils.widgets.text("file_name", "")
dbutils.widgets.text("file_schema", "")
dbutils.widgets.text("file_format", "")

In [0]:
file_name = dbutils.widgets.get("file_name")
file_schema = dbutils.widgets.get("file_schema")
file_format = dbutils.widgets.get("file_format")
file_name_pattern = f"{file_name}*"

## Ingest Data

In [0]:
df = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", file_format)\
        .option("cloudFiles.schemaLocation", f"abfss://bronze@javantatrainingstorage.dfs.core.windows.net/checkpoint_location_{file_schema}")\
        .load(f"abfss://source@javantatrainingstorage.dfs.core.windows.net/{file_schema}/{file_name_pattern}/")

df = df.withColumn("ingest_timestamp", current_timestamp())

## Write Data

In [0]:
df.writeStream.format("delta")\
        .outputMode("append")\
        .option("checkpointLocation", f"abfss://bronze@javantatrainingstorage.dfs.core.windows.net/checkpoint_location_{file_schema}")\
        .trigger(once=True)\
        .toTable(f"bronze.{file_schema}.{file_name}")
        
        